In [1]:
from pathlib import Path
import h5py
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta

import sorts
import pyant
import pyorb

base = Path('/home/danielk/git/antisat_analysis/projects/output/russian_asat')
data_path = Path('/home/danielk/data/spade/beamparks/uhf/2021.11.23/leo_bpark_2.1u_NO@uhf_extended')

In [2]:
correlation_select_file = base / '2021.11.23_uhf_correlation_plots/eiscat_uhf_selected_correlations.npy'
correlation_file = base/ '2021.11.23_uhf_correlation.pickle'

correlation_select = np.load(correlation_select_file)

with h5py.File(correlation_file, 'r') as ds:
    indecies = ds['matched_object_index'][()]
    name = ds.attrs['radar_name']

measurnment_id = np.arange(indecies.shape[1])

In [3]:
tle_file = base / '2021_11_23_spacetrack.tle'
pop = sorts.population.tle_catalog(tle_file, cartesian=False)
pop.unique()

In [18]:
events_file = data_path / '20211123_12/events.txt'
extended_files = []
with open(events_file, 'r') as fh:
    for ind, line in enumerate(fh):
        if ind < 33 or len(line.strip()) == 0:
            continue
        #filter the SNR like with the h5 files
        if float(line[69:76])**2.0 <= 33:
            print(f'skipping {line[69:76]} ratio target')
            continue
        extended_files.append(line[181:].strip())

skipping    5.3  ratio target
skipping    5.5  ratio target
skipping    5.1  ratio target
skipping    5.2  ratio target


In [19]:
spade_file = base / 'uhf/2021.11.23.h5'
with h5py.File(spade_file, 'r') as h_det:
    r = h_det['r'][()]*1e3  # km -> m, one way
    t = h_det['t'][()]  # Unix seconds
    v = h_det['v'][()]*1e3  # km/s -> m/s
    inds = np.argsort(t)
    t = t[inds]
    r = r[inds]
    v = v[inds]

    times = Time(t, format='unix', scale='utc')
    epoch = times[0]
    t = (times - epoch).sec

In [20]:
#select only from sub_folder
t0 = Time('2021-11-23 12:00:00', format='iso', scale='utc')
t1 = Time('2021-11-23 13:00:00', format='iso', scale='utc')

sub_select_inds = np.logical_and(
    times >= t0, 
    times <= t1,
)
sub_measurnment_id = measurnment_id[sub_select_inds]
sub_correlation_select = correlation_select[sub_select_inds]
sub_indecies = indecies.flatten()[sub_select_inds]

correlated_measurnment_id = sub_measurnment_id[sub_correlation_select]
correlated_object_id = sub_indecies[sub_correlation_select]

extended_files = np.array(extended_files)
correlated_extended_files = extended_files[sub_correlation_select]

In [21]:
#These are all indexed according to measurnment id
print(len(sub_select_inds))
print(len(measurnment_id))
print(len(correlation_select))
print(len(indecies.flatten()))
#these are for extended data
print(len(sub_measurnment_id))
print(len(sub_correlation_select))
print(len(sub_indecies))
print(len(extended_files))
#these are the correlated measurnments and their objects
print(len(correlated_measurnment_id))
print(len(correlated_object_id))
print(len(correlated_extended_files))

572
572
572
572
184
184
184
184
37
37
37


In [22]:
#select one
select_id = 0

meas_id = correlated_measurnment_id[select_id]
obj_id = correlated_object_id[select_id]
ext_data_file = str(correlated_extended_files[select_id])
print(meas_id)
print(obj_id)
print(ext_data_file)

146
2986
uhf_20211123_120016_600000


In [23]:
obj = pop.get_object(obj_id)
print(obj)


Space object 13801: <Time object: scale='utc' format='mjd' value=59540.06464707013>:
(b'1 13801U 81053GV  21326.06464707  .00000575  00000-0  32778-3 0  9995', b'2 13801  82.8001 248.9451 0042499 212.9904 146.8618 14.07752041984277')
Parameters: C_D=2.3, m=0.0029120161739511607, C_R=1.0, d=0.10361160122474014


### From https://www.n2yo.com/satellite/?s=13801

* NORAD ID: 13801 
* Int'l Code: 1981-053GY 
* Perigee: 841.1 km 
* Apogee: 906.4 km 
* Inclination: 82.8 ° 
* Period: 102.3 minutes 
* Semi major axis: 7244 km 
* RCS: 0.0143 m2 (small) 
* Launch date: June 4, 1981
* Source: Commonwealth of Independent States (former USSR) (CIS)
* Launch site: PLESETSK MISSILE AND SPACE COMPLEX (PKMTR)


In [24]:
from determine_beampark_rcs import load_spade_extended

data_file = data_path / f'20211123_12/{ext_data_file}/{ext_data_file}.hlist'
data = load_spade_extended(data_file)

In [25]:
radar = sorts.radars.eiscat_uhf
radar.tx[0].beam.sph_point(azimuth=90, elevation=75)